# Fallback Table Metalearning Training


In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
# install.packages("pkgcond")

In [3]:
library(dplyr)
library(data.table)
library(partykit)
library(tictoc)
library(caret)
library(e1071)
library(randomForest)
library(ranger)

#for 3d plotting
library(akima)
library(plotly)

# for prep data
library(stringr)
library(pbapply)
library(stringdist)
library(data.table)
library(dominanceanalysis)


getwd()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: grid

Loading required package: libcoin

Loading required package: mvtnorm

Loading required package: lattice

Loading required package: ggplot2

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:sta

[1] "/home/jupyter/Domains_202003/scripts/phaseII_06_fallbacksupp"

In [4]:

source('../orig/functions.R')

source('../orig/functions_models.R')

source('../phaseII_03_forest/functions_eval.R')



Attaching package: ‘tidyr’


The following object is masked from ‘package:stringdist’:

    extract




In [5]:
source('functions_metalearning.R')

In [6]:
source('functions_fallback.R')

In [7]:
# source('load_prep_data_expiry_2.R') #read in train test and predictions manually instead

# READ CSV's
# expiry_df_test_preds <- read.csv("../../data/output/datapull_20201116/expiry_df_test_preds.csv")
expiry_df_train <- read.csv("../../data/output/datapull_20201116/expiry_df_train.csv")
# expiry_df_test <- read.csv("../../data/output/datapull_20201116/expiry_df_test.csv")
metametrics_df <- read.csv("../../data/output/datapull_20201116/expiry_metametrics_df.csv")
# expiry_df
# expiry_train_df 
# expiry_test_df 
# expiry_list
# expiry_train_list
# expiry_test_list

# CREATE flags, analaze splits
## seg2_glm_fb vs agg_rf_ALL

In [8]:
# (auc_vars = grep('auc', names(metrics_df), value=TRUE))
# (l10_vars = grep('l10', names(metrics_df), value=TRUE))

auc_vars = c('auc_seg2_glm_fb','auc_agg_rf_ALL')
l10_vars = c('l10_seg2_glm_fb','l10_agg_rf_ALL')

auc_vars %in% names(metametrics_df)
l10_vars %in% names(metametrics_df)

metametrics_df <- metametrics_df %>%
    mutate (auc_win_01=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]), 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")),
            l10_win_01=sapply(apply(.[,c(l10_vars)], 
                          1, function(x) names(x)[which.max(x)]), 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
                              

[1] TRUE TRUE

[1] TRUE TRUE

In [9]:
table(metametrics_df$auc_win_01)
table(metametrics_df$l10_win_01)


 auc_agg_rf_ALL auc_seg2_glm_fb 
            762             720 


 l10_agg_rf_ALL l10_seg2_glm_fb 
            516             966 

## multiclass

In [10]:
(auc_vars = grep('auc_', names(metametrics_df), value=TRUE))
(l10_vars = grep('l10_', names(metametrics_df), value=TRUE))

(auc_vars = auc_vars[-grep("win", auc_vars)])
(l10_vars = l10_vars[-grep("win", l10_vars)])

metametrics_df <- metametrics_df %>%
    mutate (auc_win_02=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) ,
            l10_win_02=sapply(apply(.[,c(l10_vars)], 
                          1, function(x) names(x)[which.max(x)]), function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 

[1] "auc_seg2_glm"       "auc_seg_glm"        "auc_agg_glm_ALL"   
 [4] "auc_agg_glm"        "auc_seg2_rf"        "auc_seg_rf"        
 [7] "auc_agg_rf_ALL"     "auc_agg_rf"         "auc_seg2_glm_fb"   
[10] "auc_seg_glm_fb"     "auc_agg_glm_fb"     "auc_agg_glm_ALL_fb"
[13] "auc_seg2_rf_fb"     "auc_seg_rf_fb"      "auc_agg_rf_fb"     
[16] "auc_agg_rf_ALL_fb"  "auc_win_01"

[1] "l10_seg2_glm"       "l10_seg_glm"        "l10_agg_glm_ALL"   
 [4] "l10_agg_glm"        "l10_seg2_rf"        "l10_seg_rf"        
 [7] "l10_agg_rf"         "l10_agg_rf_ALL"     "l10_seg2_glm_fb"   
[10] "l10_seg_glm_fb"     "l10_agg_glm_fb"     "l10_agg_glm_ALL_fb"
[13] "l10_seg2_rf_fb"     "l10_seg_rf_fb"      "l10_agg_rf_fb"     
[16] "l10_agg_rf_ALL_fb"  "l10_win_01"

[1] "auc_seg2_glm"       "auc_seg_glm"        "auc_agg_glm_ALL"   
 [4] "auc_agg_glm"        "auc_seg2_rf"        "auc_seg_rf"        
 [7] "auc_agg_rf_ALL"     "auc_agg_rf"         "auc_seg2_glm_fb"   
[10] "auc_seg_glm_fb"     "auc_agg_glm_fb"     "auc_agg_glm_ALL_fb"
[13] "auc_seg2_rf_fb"     "auc_seg_rf_fb"      "auc_agg_rf_fb"     
[16] "auc_agg_rf_ALL_fb"

[1] "l10_seg2_glm"       "l10_seg_glm"        "l10_agg_glm_ALL"   
 [4] "l10_agg_glm"        "l10_seg2_rf"        "l10_seg_rf"        
 [7] "l10_agg_rf"         "l10_agg_rf_ALL"     "l10_seg2_glm_fb"   
[10] "l10_seg_glm_fb"     "l10_agg_glm_fb"     "l10_agg_glm_ALL_fb"
[13] "l10_seg2_rf_fb"     "l10_seg_rf_fb"      "l10_agg_rf_fb"     
[16] "l10_agg_rf_ALL_fb"

In [11]:
as.data.frame(sort(table(metametrics_df$auc_win_02), decreasing=TRUE))
as.data.frame(sort(table(metametrics_df$l10_win_02), decreasing=TRUE))

Var1,Freq
<fct>,<int>
auc_seg2_glm,487
auc_seg_rf,213
auc_seg2_rf,163
auc_seg_glm,157
auc_agg_rf_ALL,145
auc_agg_rf,116
auc_agg_glm_ALL,100
auc_agg_glm,57
auc_seg2_glm_fb,44


Var1,Freq
<fct>,<int>
l10_seg2_glm,731
l10_seg_glm,178
l10_seg2_rf,169
l10_seg_rf,132
l10_agg_glm_ALL,81
l10_agg_rf,81
l10_agg_rf_ALL,53
l10_agg_glm,36
l10_seg2_glm_fb,21


## seg2_glm_fb vs seg2_glm

In [12]:
(auc_vars = grep('auc_seg2_glm', names(metametrics_df), value=TRUE))
(l10_vars = grep('l10_seg2_glm', names(metametrics_df), value=TRUE))

metametrics_df <- metametrics_df %>%
    mutate (auc_win_03=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) ,
            l10_win_03=sapply(apply(.[,c(l10_vars)], 
                          1, function(x) names(x)[which.max(x)]), function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
                              


[1] "auc_seg2_glm"    "auc_seg2_glm_fb"

[1] "l10_seg2_glm"    "l10_seg2_glm_fb"

In [13]:
table(metametrics_df$auc_win_03)
table(metametrics_df$l10_win_03)


   auc_seg2_glm auc_seg2_glm_fb 
           1221             261 


   l10_seg2_glm l10_seg2_glm_fb 
           1306             176 

In [14]:
dim(metametrics_df)

[1] 1758   84

In [15]:
# percentage of of tld-re's replaced with FB table use
tld_registrar_excl_list = tld_registrar_excl_df(train_df = expiry_df_train)

Excluding 1706 tld-re's (80.89%), due to volume < 500 
Excluding 157 tld-re's (7.44%) associated with tld's 'pw' and 'in.net' 


In [16]:
metametrics_df %>% filter(tld_registrar_index %in% tld_registrar_excl_list) %>% nrow() / metametrics_df %>% nrow()

[1] 0.7411832

# Preprocess data - missing value impute & train/test split

In [17]:
# install.packages("missRanger")
library(missRanger)

In [18]:
metametrics_imp_df <- missRanger(metametrics_df, num.trees = 100)


Missing value imputation by random forests

  Variables to impute:		geo_maj, daydom_std, daydom_skew, daydom_kurt, sldlen_std, sldlen_skew, sldlen_kurt, gibbs_std, gibbs_skew, gibbs_kurt, pdcnt_std, pdcnt_skew, pdcnt_kurt, regarpt_std, regarpt_skew, regarpt_kurt, l10_seg2_glm, l10_seg_glm, l10_agg_glm_ALL, l10_agg_glm, l10_seg2_rf, l10_seg_rf, l10_agg_rf, l10_agg_rf_ALL, auc_seg2_glm, auc_seg_glm, auc_agg_glm_ALL, auc_agg_glm, auc_seg2_rf, auc_seg_rf, auc_agg_rf_ALL, auc_agg_rf, l10_seg2_glm_fb, l10_seg_glm_fb, l10_agg_glm_fb, l10_agg_glm_ALL_fb, l10_seg2_rf_fb, l10_seg_rf_fb, l10_agg_rf_fb, l10_agg_rf_ALL_fb, auc_seg2_glm_fb, auc_seg_glm_fb, auc_agg_glm_fb, auc_agg_glm_ALL_fb, auc_seg2_rf_fb, auc_seg_rf_fb, auc_agg_rf_fb, auc_agg_rf_ALL_fb, auc_win_01, l10_win_01, auc_win_02, l10_win_02, auc_win_03, l10_win_03
  Variables used to impute:	tld_registrar_index, geo_maj, geo_cnt, n, ren_prp, tld_cnt, tld_rat, daydom_min, daydom_max, daydom_mean, daydom_rng, daydom_std, daydom_skew, daydo

In [19]:
set.seed(123) 
smp_siz = floor(0.8*nrow(metametrics_imp_df))
train_ind = sample(seq_len(nrow(metametrics_imp_df)),size = smp_siz) 
train = metametrics_imp_df[train_ind,] 
test = metametrics_imp_df[-train_ind,]  

In [20]:
dim(train)
dim(test)

[1] 1406   84

[1] 352  84

# RF BINARY w/ win_02

In [21]:
metametrics_df %>% 
  select('auc_win_02', 'l10_win_02') %>% 
  bind_cols(
      metametrics_df %>% 
        select(-contains('auc'),
               -contains('l10'),
               -contains('tld_registrar_index'))
      ) %>% head()

,auc_win_02,l10_win_02,geo_maj,geo_cnt,n,ren_prp,tld_cnt,tld_rat,daydom_min,daydom_max,daydom_mean,daydom_rng,daydom_std,daydom_skew,daydom_kurt,sldlen_min,sldlen_max,sldlen_mean,sldlen_rng,sldlen_std,sldlen_skew,sldlen_kurt,gibbs_min,gibbs_max,gibbs_mean,gibbs_rng,gibbs_std,gibbs_skew,gibbs_kurt,pdcnt_min,pdcnt_max,pdcnt_mean,pdcnt_rng,pdcnt_std,pdcnt_skew,pdcnt_kurt,regarpt_min,regarpt_max,regarpt_mean,regarpt_rng,regarpt_std,regarpt_skew,regarpt_kurt,country_maj,region_maj,country_cnt,region_cnt
,<chr>,<chr>,<fct>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>,<int>
1,auc_seg2_glm_fb,l10_seg_rf,EU,1,11,0.54545455,1,9.090909e-02,1,5,2.181818,4,1.537412,0.9266651,-0.7811140,4,14,7.636364,10,2.907670,0.7720046,-0.3922768,1.71,9.01,4.655455,7.30,2.602016,6.453797e-01,-1.217221,1,3,1.363636,2,0.6741999,1.338593,0.3606612,15.0000,15.0,15.0000000,0.0000,0.0000000,NA,NA,Japan,Non China,1,1
2,NA,NA,EU,1,2,0.00000000,1,5.000000e-01,1,1,1.000000,0,0.000000,NA,NA,5,8,6.500000,3,2.121320,0.0000000,-2.7500000,3.28,8.62,5.950000,5.34,3.775950,9.898575e-17,-2.750000,1,1,1.000000,0,0.0000000,NA,NA,15.0000,15.0,15.0000000,0.0000,0.0000000,NA,NA,United States,Non China,1,1
3,auc_agg_glm_ALL,l10_agg_glm_ALL,EU,1,8,0.50000000,1,1.250000e-01,1,1,1.000000,0,0.000000,NA,NA,4,15,8.875000,11,3.522884,0.4848693,-1.1434523,2.00,6.09,4.396250,4.09,1.360167,-3.531195e-01,-1.301177,1,1,1.000000,0,0.0000000,NA,NA,3.0000,5.0,4.0000000,2.0000,1.0690450,0.000000,-2.234375,China,China,1,1
4,auc_seg2_glm_fb,l10_seg_rf,EU,1,47,0.06382979,1,2.127660e-02,1,119,82.872340,118,46.011632,-0.8738906,-0.9645903,4,61,9.446809,57,8.334863,5.0179710,28.2025571,0.28,17.92,4.695745,17.64,3.112366,1.526987e+00,4.826982,1,3,1.191489,2,0.4490714,2.194541,4.1889225,0.0001,15.0,0.8511511,14.9999,2.6207801,3.740087,15.752244,United States,Non China,1,1
5,auc_seg2_rf,l10_seg2_rf,EU,1,25,0.20000000,1,4.000000e-02,1,14,5.040000,13,5.020624,1.0166170,-0.7554981,4,19,7.080000,15,3.451087,1.9087776,3.6548313,0.59,100.00,9.743600,99.41,19.035942,4.242864e+00,17.222688,1,10,1.520000,9,1.8055470,4.127909,16.4692321,0.5000,0.5,0.5000000,0.0000,0.0000000,NA,NA,China,China,1,1
6,auc_agg_rf_ALL,l10_agg_rf_ALL,China,1,35046,0.02014495,1,2.853393e-05,19,7163,1299.015380,7144,1600.896587,2.5691646,6.0230231,3,63,6.947155,60,3.018534,2.7839889,17.5227635,0.00,100.00,5.653614,100.00,19.616978,4.525337e+00,18.821500,1,400,23.168949,399,52.1502807,3.559110,14.2005513,0.0001,5.0,0.7338244,4.9999,0.1809528,3.717292,137.140359,United States,Non China,1,1


### AUC 

In [22]:
library(ranger) 

In [36]:
#Compute weights to balance the RF
Y = train$auc_win_02
w <- 1/table(Y)
w <- w/sum(w)

weights <- rep(0, length(Y))

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

      Y
       auc_agg_glm auc_agg_glm_ALL auc_agg_rf auc_agg_rf_ALL auc_seg_glm
  0.02           0               0          0              0           0
  0.06           0               0          0              0           0
  0.07           0               0          0              0           0
  0.08           0               0          0            118         128
  0.11           0               0         92              0           0
  0.12           0              81          0              0           0
  0.21          47               0          0              0           0
  0.26           0               0          0              0           0
      Y
       auc_seg_rf auc_seg2_glm auc_seg2_glm_fb auc_seg2_rf
  0.02          0          584               0           0
  0.06        171            0               0           0
  0.07          0            0               0         148
  0.08          0            0               0           0
  0.11          0            0  

In [37]:
model <- ranger(formula         = auc_win_02 ~ ., 
                data            = train %>% 
                                    select('auc_win_02') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [38]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (auc_win_02=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 

In [39]:
data.frame(sort(prop.table(table(pred$auc_win_02))))

Var1,Freq
<fct>,<dbl>
auc_agg_glm,0.005681818
auc_seg2_glm_fb,0.008522727
auc_agg_glm_ALL,0.034090909
auc_agg_rf,0.051136364
auc_agg_rf_ALL,0.105113636
auc_seg_glm,0.110795455
auc_seg2_rf,0.110795455
auc_seg_rf,0.173295455
auc_seg2_glm,0.400568182


In [41]:
xtab <- table(test$auc_win_02, pred$auc_win_02)
xtab

                 
                  auc_agg_glm auc_agg_glm_ALL auc_agg_rf auc_agg_rf_ALL
  auc_agg_glm               0               0          1              1
  auc_agg_glm_ALL           0               1          1              1
  auc_agg_rf                0               3          5              2
  auc_agg_rf_ALL            0               2          2              8
  auc_seg_glm               1               2          1              3
  auc_seg_rf                0               1          5             11
  auc_seg2_glm              1               2          2              8
  auc_seg2_glm_fb           0               0          0              1
  auc_seg2_rf               0               1          1              2
                 
                  auc_seg_glm auc_seg_rf auc_seg2_glm auc_seg2_glm_fb
  auc_agg_glm               2          4            1               0
  auc_agg_glm_ALL           6          1            5               1
  auc_agg_rf                4     

In [42]:
levels(factor(test$auc_win_02))
levels(factor(pred$auc_win_02))

[1] "auc_agg_glm"     "auc_agg_glm_ALL" "auc_agg_rf"      "auc_agg_rf_ALL" 
[5] "auc_seg_glm"     "auc_seg_rf"      "auc_seg2_glm"    "auc_seg2_glm_fb"
[9] "auc_seg2_rf"

[1] "auc_agg_glm"     "auc_agg_glm_ALL" "auc_agg_rf"      "auc_agg_rf_ALL" 
[5] "auc_seg_glm"     "auc_seg_rf"      "auc_seg2_glm"    "auc_seg2_glm_fb"
[9] "auc_seg2_rf"

In [43]:
unique(train$auc_win_02)
unique(test$auc_win_02)

[1] "auc_seg_rf"      "auc_seg2_glm"    "auc_agg_glm"     "auc_agg_rf_ALL" 
[5] "auc_seg2_rf"     "auc_agg_glm_ALL" "auc_agg_rf"      "auc_seg2_glm_fb"
[9] "auc_seg_glm"

[1] "auc_agg_glm_ALL" "auc_seg2_glm"    "auc_seg_rf"      "auc_seg_glm"    
[5] "auc_agg_rf"      "auc_seg2_rf"     "auc_seg2_glm_fb" "auc_agg_rf_ALL" 
[9] "auc_agg_glm"

In [44]:
confusionMatrix(data=factor(test$auc_win_02,
                           levels=levels(factor(pred$auc_win_02))), reference=factor(pred$auc_win_02))

Confusion Matrix and Statistics

                 Reference
Prediction        auc_agg_glm auc_agg_glm_ALL auc_agg_rf auc_agg_rf_ALL
  auc_agg_glm               0               0          1              1
  auc_agg_glm_ALL           0               1          1              1
  auc_agg_rf                0               3          5              2
  auc_agg_rf_ALL            0               2          2              8
  auc_seg_glm               1               2          1              3
  auc_seg_rf                0               1          5             11
  auc_seg2_glm              1               2          2              8
  auc_seg2_glm_fb           0               0          0              1
  auc_seg2_rf               0               1          1              2
                 Reference
Prediction        auc_seg_glm auc_seg_rf auc_seg2_glm auc_seg2_glm_fb
  auc_agg_glm               2          4            1               0
  auc_agg_glm_ALL           6          1            5

In [45]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

,importance(model)
,<dbl>
n,33.27559
ren_prp,26.13155
gibbs_rng,25.72643
sldlen_kurt,23.26212
daydom_skew,22.09165
gibbs_kurt,21.52359
sldlen_rng,20.53707
daydom_kurt,19.36542
daydom_rng,18.33922


In [46]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15) %>% row.names()

[1] "n"           "ren_prp"     "gibbs_rng"   "sldlen_kurt" "daydom_skew"
 [6] "gibbs_kurt"  "sldlen_rng"  "daydom_kurt" "daydom_rng"  "pdcnt_skew" 
[11] "daydom_mean" "sldlen_skew" "sldlen_std"  "daydom_max"  "gibbs_std"

### L10

In [47]:
library(ranger) 

In [48]:
#Compute weights to balance the RF
Y = train$l10_win_02
w <- 1/table(Y)
w <- w/sum(w)

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

      Y
       l10_agg_glm l10_agg_glm_ALL l10_agg_rf l10_agg_rf_ALL l10_seg_glm
  0.01           0               0          0              0           0
  0.04           0               0          0              0         144
  0.05           0               0          0              0           0
  0.08           0               0         70              0           0
  0.09           0              65          0              0           0
  0.14           0               0          0             42           0
  0.18          32               0          0              0           0
  0.38           0               0          0              0           0
      Y
       l10_seg_rf l10_seg2_glm l10_seg2_glm_fb l10_seg2_rf
  0.01          0          786               0           0
  0.04          0            0               0         145
  0.05        107            0               0           0
  0.08          0            0               0           0
  0.09          0            0  

In [49]:
model <- ranger(formula         = l10_win_02 ~ ., 
                data            = train %>% 
                                    select('l10_win_02') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [50]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
vars <- names(pred)
pred <- pred %>%
    mutate (l10_win_02=sapply(apply(.[,c(vars)], 
                          1, function(x) names(x)[which.max(x)]) , function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

,l10_agg_glm,l10_agg_glm_ALL,l10_agg_rf,l10_agg_rf_ALL,l10_seg_glm,l10_seg_rf,l10_seg2_glm,l10_seg2_glm_fb,l10_seg2_rf,l10_win_02
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.046107143,0.12972857,0.024352381,0.0330666667,0.09808095,0.047103175,0.5850310,0.0164507937,0.020079365,l10_seg2_glm
2,0.006895238,0.07811429,0.066870635,0.0394857143,0.07510952,0.262149206,0.1455365,0.0059603175,0.319878571,l10_seg2_rf
3,0.034887302,0.02626746,0.001844444,0.0000000000,0.03827540,0.015524603,0.8818008,0.0000000000,0.001400000,l10_seg2_glm
4,0.028417460,0.05007857,0.032798413,0.0117817460,0.13967619,0.062572222,0.5872437,0.0292071429,0.058224603,l10_seg2_glm
5,0.007315873,0.04744762,0.014799206,0.0002857143,0.13516825,0.008557143,0.7832048,0.0005833333,0.002638095,l10_seg2_glm
6,0.011672222,0.04377381,0.051314286,0.0542515873,0.08328413,0.119780952,0.4895238,0.0059293651,0.140469841,l10_seg2_glm


In [51]:
xtab <- table(test$l10_win_02, pred$l10_win_02)
xtab

                 
                  l10_agg_rf l10_agg_rf_ALL l10_seg_glm l10_seg_rf l10_seg2_glm
  l10_agg_glm              1              0           0          0            3
  l10_agg_glm_ALL          0              0           5          1            7
  l10_agg_rf               0              1           4          1            4
  l10_agg_rf_ALL           0              0           1          2            2
  l10_seg_glm              0              0           4          1           17
  l10_seg_rf               1              0           4          8            5
  l10_seg2_glm             1              0           6          3          188
  l10_seg2_glm_fb          0              0           2          0            3
  l10_seg2_rf              0              0           1          3            7
                 
                  l10_seg2_rf
  l10_agg_glm               0
  l10_agg_glm_ALL           3
  l10_agg_rf                2
  l10_agg_rf_ALL            6
  l10_seg_glm 

In [52]:
confusionMatrix(data=factor(test$l10_win_02), reference=factor(pred$l10_win_02, levels=levels(factor(test$l10_win_02))))

Confusion Matrix and Statistics

                 Reference
Prediction        l10_agg_glm l10_agg_glm_ALL l10_agg_rf l10_agg_rf_ALL
  l10_agg_glm               0               0          1              0
  l10_agg_glm_ALL           0               0          0              0
  l10_agg_rf                0               0          0              1
  l10_agg_rf_ALL            0               0          0              0
  l10_seg_glm               0               0          0              0
  l10_seg_rf                0               0          1              0
  l10_seg2_glm              0               0          1              0
  l10_seg2_glm_fb           0               0          0              0
  l10_seg2_rf               0               0          0              0
                 Reference
Prediction        l10_seg_glm l10_seg_rf l10_seg2_glm l10_seg2_glm_fb
  l10_agg_glm               0          0            3               0
  l10_agg_glm_ALL           5          1            7

In [53]:
table(test$l10_win_02)


    l10_agg_glm l10_agg_glm_ALL      l10_agg_rf  l10_agg_rf_ALL     l10_seg_glm 
              4              16              12              11              34 
     l10_seg_rf    l10_seg2_glm l10_seg2_glm_fb     l10_seg2_rf 
             26             218               6              25 

In [54]:
prop.table(table(pred$l10_win_02))


    l10_agg_rf l10_agg_rf_ALL    l10_seg_glm     l10_seg_rf   l10_seg2_glm 
   0.008522727    0.002840909    0.076704545    0.053977273    0.670454545 
   l10_seg2_rf 
   0.187500000 

In [55]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

,importance(model)
,<dbl>
n,34.52838
sldlen_kurt,26.26625
ren_prp,23.45355
sldlen_rng,19.89118
gibbs_rng,19.76998
daydom_rng,18.04565
pdcnt_skew,16.39951
gibbs_max,16.10392
daydom_skew,15.20988


In [56]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15) %>% row.names()

[1] "n"           "sldlen_kurt" "ren_prp"     "sldlen_rng"  "gibbs_rng"  
 [6] "daydom_rng"  "pdcnt_skew"  "gibbs_max"   "daydom_skew" "sldlen_skew"
[11] "daydom_max"  "gibbs_kurt"  "pdcnt_kurt"  "daydom_kurt" "gibbs_skew"

# RF BINARY w/ auc_win_03

## RF binary auc

In [ ]:
library(ranger) 

In [62]:
#Compute weights to balance the RF
Y = train$auc_win_03
w <- 1/table(Y)
w <- w/sum(w)


weights <- rep(0, length(Y))

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

      Y
       auc_seg2_glm auc_seg2_glm_fb
  0.15         1196               0
  0.85            0             210

In [63]:
model <- ranger(formula         = auc_win_03 ~ ., 
                data            = train %>% 
                                    select('auc_win_03') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [64]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (auc_win_03=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

,auc_seg2_glm,auc_seg2_glm_fb,auc_win_03
,<dbl>,<dbl>,<chr>
1,0.6195817,0.38041825,auc_seg2_glm
2,0.8273563,0.17264365,auc_seg2_glm
3,0.9589754,0.04102460,auc_seg2_glm
4,0.4912294,0.50877063,auc_seg2_glm_fb
5,0.9415603,0.05843968,auc_seg2_glm
6,0.9649071,0.03509286,auc_seg2_glm


In [65]:
prop.table(table(pred$auc_win_03))


   auc_seg2_glm auc_seg2_glm_fb 
     0.94602273      0.05397727 

In [66]:
xtab <- table(test$auc_win_01, pred$auc_win_03)
xtab

                 
                  auc_seg2_glm auc_seg2_glm_fb
  auc_agg_rf_ALL           151              10
  auc_seg2_glm_fb          182               9

In [67]:
confusionMatrix(data=factor(test$auc_win_03), 
                reference=factor(pred$auc_win_03, levels=levels(factor(test$auc_win_03))))

Confusion Matrix and Statistics

                 Reference
Prediction        auc_seg2_glm auc_seg2_glm_fb
  auc_seg2_glm             291              10
  auc_seg2_glm_fb           42               9
                                          
               Accuracy : 0.8523          
                 95% CI : (0.8108, 0.8877)
    No Information Rate : 0.946           
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.1937          
                                          
 Mcnemar's Test P-Value : 1.716e-05       
                                          
            Sensitivity : 0.8739          
            Specificity : 0.4737          
         Pos Pred Value : 0.9668          
         Neg Pred Value : 0.1765          
             Prevalence : 0.9460          
         Detection Rate : 0.8267          
   Detection Prevalence : 0.8551          
      Balanced Accuracy : 0.6738          
                         

In [68]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

,importance(model)
,<dbl>
ren_prp,15.781213
n,14.832819
daydom_std,11.246247
country_maj,10.826304
gibbs_rng,9.264955
daydom_skew,9.097403
pdcnt_kurt,8.097096
gibbs_skew,8.096050
daydom_mean,8.066799


In [69]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(5) %>% row.names

[1] "ren_prp"     "n"           "daydom_std"  "country_maj" "gibbs_rng"

## RF binary l10

In [70]:
library(ranger) 

In [71]:
#Compute weights to balance the RF
Y = train$l10_win_03
w <- 1/table(Y)
w <- w/sum(w)

weights <- rep(0, length(Y))

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

     Y
      l10_seg2_glm l10_seg2_glm_fb
  0.1         1268               0
  0.9            0             138

In [72]:
model <- ranger(formula         = l10_win_03 ~ ., 
                data            = train %>% 
                                    select('l10_win_03') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [73]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (l10_win_03=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

,l10_seg2_glm,l10_seg2_glm_fb,l10_win_03
,<dbl>,<dbl>,<chr>
1,0.9007310,0.09926905,l10_seg2_glm
2,0.9673992,0.03260079,l10_seg2_glm
3,0.9466183,0.05338175,l10_seg2_glm
4,0.7222183,0.27778175,l10_seg2_glm
5,0.9369825,0.06301746,l10_seg2_glm
6,0.9742849,0.02571508,l10_seg2_glm


In [74]:
prop.table(table(pred$l10_win_03))


   l10_seg2_glm l10_seg2_glm_fb 
     0.98863636      0.01136364 

In [75]:
xtab <- table(test$l10_win_03, pred$l10_win_03)
xtab

                 
                  l10_seg2_glm l10_seg2_glm_fb
  l10_seg2_glm             312               2
  l10_seg2_glm_fb           36               2

In [76]:
confusionMatrix(data=factor(test$l10_win_03), 
                reference=factor(pred$l10_win_03, levels=levels(factor(test$l10_win_03))))

Confusion Matrix and Statistics

                 Reference
Prediction        l10_seg2_glm l10_seg2_glm_fb
  l10_seg2_glm             312               2
  l10_seg2_glm_fb           36               2
                                          
               Accuracy : 0.892           
                 95% CI : (0.8548, 0.9225)
    No Information Rate : 0.9886          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.0762          
                                          
 Mcnemar's Test P-Value : 8.636e-08       
                                          
            Sensitivity : 0.89655         
            Specificity : 0.50000         
         Pos Pred Value : 0.99363         
         Neg Pred Value : 0.05263         
             Prevalence : 0.98864         
         Detection Rate : 0.88636         
   Detection Prevalence : 0.89205         
      Balanced Accuracy : 0.69828         
                         

In [139]:
length((test$l10_win_03))
length((pred$l10_win_03))

[1] 352

[1] 352

In [ ]:
prop.table(table(test$l10_win_03))
prop.table(table(test$auc_win_03))

In [ ]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

In [ ]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(5) %>% row.names()

# RF BINARY w/ auc_win_03

## RF binary auc

In [ ]:
library(ranger) 

In [ ]:
#Compute weights to balance the RF
Y = train$auc_win_03
w <- 1/table(Y)
w <- w/sum(w)

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

In [121]:
model <- ranger(formula         = auc_win_01 ~ ., 
                data            = train %>% 
                                    select('auc_win_01') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [122]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (auc_win_01=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

,auc_agg_rf_ALL,auc_seg2_glm_fb,auc_win_01
,<dbl>,<dbl>,<chr>
1,0.4868460,0.5131540,auc_seg2_glm_fb
2,0.6879087,0.3120913,auc_agg_rf_ALL
3,0.1507087,0.8492913,auc_seg2_glm_fb
4,0.5063397,0.4936603,auc_agg_rf_ALL
5,0.2291079,0.7708921,auc_seg2_glm_fb
6,0.7603794,0.2396206,auc_agg_rf_ALL


In [123]:
prop.table(table(pred$auc_win_01))


 auc_agg_rf_ALL auc_seg2_glm_fb 
      0.5795455       0.4204545 

In [124]:
xtab <- table(test$auc_win_01, pred$auc_win_01)
xtab

                 
                  auc_agg_rf_ALL auc_seg2_glm_fb
  auc_agg_rf_ALL             135              30
  auc_seg2_glm_fb             69             118

In [125]:
confusionMatrix(data=factor(test$auc_win_01), 
                reference=factor(pred$auc_win_01, levels=levels(factor(test$auc_win_01))))

Confusion Matrix and Statistics

                 Reference
Prediction        auc_agg_rf_ALL auc_seg2_glm_fb
  auc_agg_rf_ALL             135              30
  auc_seg2_glm_fb             69             118
                                          
               Accuracy : 0.7188          
                 95% CI : (0.6687, 0.7651)
    No Information Rate : 0.5795          
    P-Value [Acc > NIR] : 4.33e-08        
                                          
                  Kappa : 0.443           
                                          
 Mcnemar's Test P-Value : 0.0001339       
                                          
            Sensitivity : 0.6618          
            Specificity : 0.7973          
         Pos Pred Value : 0.8182          
         Neg Pred Value : 0.6310          
             Prevalence : 0.5795          
         Detection Rate : 0.3835          
   Detection Prevalence : 0.4688          
      Balanced Accuracy : 0.7295          
                   

In [128]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

,importance(model)
,<dbl>
n,35.69194
sldlen_kurt,27.26609
gibbs_kurt,23.45892
daydom_rng,21.76428
gibbs_rng,19.67531
daydom_kurt,18.49228
ren_prp,15.96621
country_maj,14.90314
sldlen_skew,14.84797


In [129]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(5) %>% row.names

[1] "n"           "sldlen_kurt" "gibbs_kurt"  "daydom_rng"  "gibbs_rng"

## RF binary l10

In [ ]:
library(ranger) 

In [130]:
#Compute weights to balance the RF
Y = train$l10_win_01
w <- 1/table(Y)
w <- w/sum(w)

for (model in unique(Y)){
    weights[Y==model] <- w[model]
}
table(round(weights,2), Y)

     Y
      l10_agg_rf_ALL l10_seg2_glm_fb
  0.3              0             988
  0.7            418               0

In [131]:
model <- ranger(formula         = l10_win_01 ~ ., 
                data            = train %>% 
                                    select('l10_win_01') %>% 
                                    bind_cols(
                                        train %>% 
                                        select(-contains('auc'),-contains('l10'),-'tld_registrar_index', -'tld_rat')), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE,
               case.weights=weights)

In [134]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (l10_win_01=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

,l10_agg_rf_ALL,l10_seg2_glm_fb,l10_win_01
,<dbl>,<dbl>,<chr>
1,0.16256587,0.8374341,l10_seg2_glm_fb
2,0.67128968,0.3287103,l10_agg_rf_ALL
3,0.08287063,0.9171294,l10_seg2_glm_fb
4,0.40130079,0.5986992,l10_seg2_glm_fb
5,0.21686349,0.7831365,l10_seg2_glm_fb
6,0.47004286,0.5299571,l10_seg2_glm_fb


In [136]:
prop.table(table(pred$l10_win_01))


 l10_agg_rf_ALL l10_seg2_glm_fb 
         0.3125          0.6875 

In [137]:
xtab <- table(test$l10_win_01, pred$l10_win_01)
xtab

                 
                  l10_agg_rf_ALL l10_seg2_glm_fb
  l10_agg_rf_ALL              64              39
  l10_seg2_glm_fb             46             203

In [138]:
confusionMatrix(data=factor(test$l10_win_01), 
                reference=factor(pred$l10_win_01, levels=levels(factor(test$l10_win_01))))

Confusion Matrix and Statistics

                 Reference
Prediction        l10_agg_rf_ALL l10_seg2_glm_fb
  l10_agg_rf_ALL              64              39
  l10_seg2_glm_fb             46             203
                                          
               Accuracy : 0.7585          
                 95% CI : (0.7103, 0.8023)
    No Information Rate : 0.6875          
    P-Value [Acc > NIR] : 0.002017        
                                          
                  Kappa : 0.4281          
                                          
 Mcnemar's Test P-Value : 0.515181        
                                          
            Sensitivity : 0.5818          
            Specificity : 0.8388          
         Pos Pred Value : 0.6214          
         Neg Pred Value : 0.8153          
             Prevalence : 0.3125          
         Detection Rate : 0.1818          
   Detection Prevalence : 0.2926          
      Balanced Accuracy : 0.7103          
                   

In [139]:
length((test$l10_win_01))
length((pred$l10_win_01))

[1] 352

[1] 352

In [140]:
prop.table(table(test$l10_win_01))
prop.table(table(test$auc_win_01))


 l10_agg_rf_ALL l10_seg2_glm_fb 
      0.2926136       0.7073864 


 auc_agg_rf_ALL auc_seg2_glm_fb 
        0.46875         0.53125 

In [141]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(15)

,importance(model)
,<dbl>
n,29.68940
daydom_rng,23.76639
daydom_max,19.52655
sldlen_kurt,18.10491
daydom_mean,16.28520
ren_prp,15.05664
gibbs_rng,14.48129
country_maj,14.04315
sldlen_rng,13.72672


In [142]:
as.data.frame(importance(model)) %>% arrange(desc(importance(model))) %>% head(5) %>% row.names()

[1] "n"           "daydom_rng"  "daydom_max"  "sldlen_kurt" "daydom_mean"

# Assign Model based on Binary prediction for AUC

In [ ]:
model <- ranger(formula         = auc_win_01 ~ ., 
                data            = subset(train, select = -c(l10_win_01) ), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE)

In [ ]:
pred <- predict(model, 
                data = test,
                type="response")$predictions

In [ ]:
pred <- as.data.frame(pred)
auc_vars <- names(pred)
pred <- pred %>%
    mutate (auc_win_01=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

In [ ]:
test_pred <- cbind(test,pred)
# test_pred <- test_pred %>% select()
dim(test)
dim(pred)
dim(test_pred)
head(test_pred)

# Calc metrics

## generate lookup for tld-reg model

In [ ]:
head(expiry_test_meta)

In [ ]:
expiry_test_meta_test <- expiry_test_meta[-train_ind,]

dim(pred)
dim(expiry_test_meta_test)

expiry_test_meta_test <- cbind(expiry_test_meta_test, pred)
expiry_test_meta_test <- expiry_test_meta_test %>% select(tld_registrar_index, auc_win_01)

expiry_test_meta_test %>% head()

## assign prediction based on lookup

In [ ]:
expiry_test_predictions_assign <- merge(expiry_test_predictions, 
                                       expiry_test_meta_test, 
                                       on='tld_registrar_index',
                                       all=TRUE)
dim(expiry_test_predictions_assign)
dim(expiry_test_predictions)
head(expiry_test_predictions_assign)

In [ ]:
# metalearning trained on 80% subset of 20% subset of expiry
# tested on 20% subset of 20% of expiry

table(expiry_test_predictions_assign$auc_win_01)/nrow(expiry_test_predictions_assign)
sum(is.na(expiry_test_predictions_assign$auc_win_01))/nrow(expiry_test_predictions_assign)

In [ ]:
# remove all NAs
# ie elements of test which were used to train the model
expiry_test_predictions_assign <- expiry_test_predictions_assign %>% filter(!is.na(auc_win_01))
dim(expiry_test_predictions_assign)
sum(is.na(expiry_test_predictions_assign$auc_win_01))/nrow(expiry_test_predictions_assign)
head(expiry_test_predictions_assign)

In [ ]:
expiry_test_predictions_assign$auc_win_pred <- 
    (expiry_test_predictions_assign$auc_win_01=='auc_seg2_glm_fb')*expiry_test_predictions_assign$pred_df_seg2_glm_fb+
    (expiry_test_predictions_assign$auc_win_01=='auc_agg_rf_fb')*expiry_test_predictions_assign$pred_df_agg_rf_fb

head(expiry_test_predictions_assign %>% select(pred_df_seg2_glm_fb,pred_df_agg_rf_fb,auc_win_01,auc_win_pred))

In [ ]:

lift_list <- list()
auc_list <- list()

fallback_lift <- chart_lift(pred_df=expiry_test_predictions_assign,
                        dep_var = "renewal_status",
                        pred_var = "auc_win_pred") 

lift_list[["auc_win_pred"]] <- fallback_lift
auc_list[["auc_win_pred"]] <- calc_auc(lift_df=fallback_lift)

models <- c('pred_df_seg2_glm_fb','pred_df_agg_rf_fb')


for (model in models){
    model_lift <- chart_lift(pred_df=expiry_test_predictions_assign,
                        dep_var = "renewal_status",
                        pred_var = model) 
    
    fallback_lift <- merge(fallback_lift, model_lift, by=c("P"),all= TRUE, 
                           suffixes = c("",
                                        paste("_",gsub("pred_df_", "", model)) ))
    
    lift_list[[model]] <- model_lift
    auc_list[[model]] <- calc_auc(lift_df=model_lift)
}

lift_df <- fallback_lift

In [ ]:
length(lift_list)
names(lift_list)
sapply(lift_list, dim)
lift_list["auc_win_pred"]

In [ ]:
prop.table(table(expiry_test_predictions_assign$renewal_status))

In [ ]:
plot_multigains <- function (lift_df_list=list(seg_glm = lift_df, 
                                               ben_dtr = lift_df_2),
                             auc_list = list(seg_glm = auc, 
                                             ben_dtr=auc_2),
                             prop_positive = 0.1194494) {
  
  optimal_df <- data.frame( "P"=c(0,prop_positive,1.0),
                            "actu_renwd2"=c(NA,NA,NA),
                            "gain"=c(0,1.0,1.0),
                            "lift"=c(NA,NA,NA))
  
  lift_df_list <- lapply(lift_df_list, function(df) {
    df <- df %>%
      add_row(P = 0, gain =0) %>%
      arrange(P)
  })

  auc_lift_list = paste(lapply(auc_list, round, 3),sapply(lift_df_list, function(x) round(x[2,"lift"],2)), sep=", ")
  
  auc_map = paste(names(lift_df_list),auc_lift_list, sep=" = ")[1:length(lift_df_list)]
  
  
  gains_plot <- ggplot(NULL, aes(P,  gain)) +
    geom_line(data = lift_df_list[[1]] %>% slice(1, n())) +
    
    list(geom_line(data=optimal_df), geom_point(data=optimal_df)) +
    
    scale_y_continuous(breaks = seq(0, 1, by = .1), limits = c(0,1)) +
    scale_x_continuous(breaks = seq(0, 1, by = .1)) +
    labs(title = "Cumulative Gains Plot",
         y = "Cumulative Gain",
         x = "Percentile")+
    theme(text = element_text(size=20))
  
  for(i in seq(length(lift_df_list))){
    name = names(lift_df_list)[[i]]
    df = lift_df_list[[i]]
    auc = auc_list[[i]]
    gains_plot <- gains_plot + list(geom_line(data=df), 
                                    geom_point(data=df))+ 
      annotate("text", x = .4, y = .9-i*.075, hjust = 0, label = auc_map[[i]], size = 5)
  }
  
  
  return(gains_plot)
}


In [ ]:
plot_multigains(            lift_df_list=lift_list,
                             auc_list = auc_list,
                             prop_positive = 0.1194494) 

In [ ]:
dim(expiry_test_predictions_assign)

In [ ]:
lift_df

In [ ]:
as.data.frame(auc_list)

# Assign Model based on Binary prediction for L10

In [ ]:
model <- ranger(formula         = l10_win_01 ~ ., 
                data            = subset(train, select = -c(auc_win_01) ), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE)

In [ ]:
pred <- predict(model, 
                data = test,
                type="response")$predictions
head(pred)

In [ ]:
pred <- as.data.frame(pred)
(auc_vars <- names(pred))
pred <- pred %>%
    mutate (l10_win_01=sapply(apply(.[,c(auc_vars)], 
                          1, function(x) names(x)[which.max(x)]) , 
                                    function(s) if (length(s) == 0) NA else paste(s, collapse = " ")) 
            ) 
head(pred)

In [ ]:
test_pred <- cbind(test,pred)
# test_pred <- test_pred %>% select()
dim(test)
dim(pred)
dim(test_pred)
head(test_pred)

# Calc metrics

## generate lookup for tld-reg model

In [ ]:
head(expiry_test_meta)

In [ ]:
expiry_test_meta_test <- expiry_test_meta[-train_ind,]

dim(pred)
dim(expiry_test_meta_test)

expiry_test_meta_test <- cbind(expiry_test_meta_test, pred)
expiry_test_meta_test <- expiry_test_meta_test %>% select(tld_registrar_index, l10_win_01)

expiry_test_meta_test %>% head()

## assign prediction based on lookup

In [ ]:
expiry_test_predictions_assign <- merge(expiry_test_predictions, 
                                       expiry_test_meta_test, 
                                       on='tld_registrar_index',
                                       all=TRUE)
dim(expiry_test_predictions_assign)
dim(expiry_test_predictions)
head(expiry_test_predictions_assign)

In [ ]:
# metalearning trained on 80% subset of 20% subset of expiry
# tested on 20% subset of 20% of expiry

table(expiry_test_predictions_assign$l10_win_01)/nrow(expiry_test_predictions_assign)
sum(is.na(expiry_test_predictions_assign$l10_win_01))/nrow(expiry_test_predictions_assign)

In [ ]:
# remove all NAs
# ie elements of test which were used to train the model
expiry_test_predictions_assign <- expiry_test_predictions_assign %>% filter(!is.na(l10_win_01))
dim(expiry_test_predictions_assign)
sum(is.na(expiry_test_predictions_assign$l10_win_01))/nrow(expiry_test_predictions_assign)
head(expiry_test_predictions_assign)

In [ ]:
expiry_test_predictions_assign$l10_win_pred <- 
    (expiry_test_predictions_assign$l10_win_01=='l10_seg2_glm_fb')*expiry_test_predictions_assign$pred_df_seg2_glm_fb+
    (expiry_test_predictions_assign$l10_win_01=='l10_agg_rf_fb')*expiry_test_predictions_assign$pred_df_agg_rf_fb

head(expiry_test_predictions_assign %>% select(pred_df_seg2_glm_fb,pred_df_agg_rf_fb,l10_win_01,l10_win_pred))

In [ ]:

lift_list <- list()
auc_list <- list()

fallback_lift <- chart_lift(pred_df=expiry_test_predictions_assign,
                        dep_var = "renewal_status",
                        pred_var = "l10_win_pred") 

lift_list[["l10_win_pred"]] <- fallback_lift
auc_list[["l10_win_pred"]] <- calc_auc(lift_df=fallback_lift)

models <- c('pred_df_seg2_glm_fb','pred_df_agg_rf_fb')


for (model in models){
    model_lift <- chart_lift(pred_df=expiry_test_predictions_assign,
                        dep_var = "renewal_status",
                        pred_var = model) 
    
    fallback_lift <- merge(fallback_lift, model_lift, by=c("P"),all= TRUE, 
                           suffixes = c("",
                                        paste("_",gsub("pred_df_", "", model)) ))
    
    lift_list[[model]] <- model_lift
    auc_list[[model]] <- calc_auc(lift_df=model_lift)
}

lift_df <- fallback_lift

In [ ]:
length(lift_list)
names(lift_list)
sapply(lift_list, dim)
lift_list["auc_win_pred"]

In [ ]:
prop.table(table(expiry_test_predictions_assign$renewal_status))

In [ ]:
plot_multigains <- function (lift_df_list=list(seg_glm = lift_df, 
                                               ben_dtr = lift_df_2),
                             auc_list = list(seg_glm = auc, 
                                             ben_dtr=auc_2),
                             prop_positive = 0.1194494) {
  
  optimal_df <- data.frame( "P"=c(0,prop_positive,1.0),
                            "actu_renwd2"=c(NA,NA,NA),
                            "gain"=c(0,1.0,1.0),
                            "lift"=c(NA,NA,NA))
  
  lift_df_list <- lapply(lift_df_list, function(df) {
    df <- df %>%
      add_row(P = 0, gain =0) %>%
      arrange(P)
  })

  auc_lift_list = paste(lapply(auc_list, round, 3),sapply(lift_df_list, function(x) round(x[2,"lift"],2)), sep=", ")
  
  auc_map = paste(names(lift_df_list),auc_lift_list, sep=" = ")[1:length(lift_df_list)]
  
  
  gains_plot <- ggplot(NULL, aes(P,  gain)) +
    geom_line(data = lift_df_list[[1]] %>% slice(1, n())) +
    
    list(geom_line(data=optimal_df), geom_point(data=optimal_df)) +
    
    scale_y_continuous(breaks = seq(0, 1, by = .1), limits = c(0,1)) +
    scale_x_continuous(breaks = seq(0, 1, by = .1)) +
    labs(title = "Cumulative Gains Plot",
         y = "Cumulative Gain",
         x = "Percentile")+
    theme(text = element_text(size=20))
  
  for(i in seq(length(lift_df_list))){
    name = names(lift_df_list)[[i]]
    df = lift_df_list[[i]]
    auc = auc_list[[i]]
    gains_plot <- gains_plot + list(geom_line(data=df), 
                                    geom_point(data=df))+ 
      annotate("text", x = .4, y = .9-i*.075, hjust = 0, label = auc_map[[i]], size = 5)
  }
  
  
  return(gains_plot)
}


In [ ]:
plot_multigains(            lift_df_list=lift_list,
                             auc_list = auc_list,
                             prop_positive = 0.1194494) 

In [ ]:
dim(expiry_test_predictions_assign)

In [ ]:
lift_df

In [ ]:
as.data.frame(auc_list)

# Examining how variables of high importance differ between classes in training & test data

In [ ]:
model <- ranger(formula         = auc_win_01 ~ ., 
                data            = subset(train, select = -c(l10_win_01) ), 
                importance = 'impurity', 
                num.trees       = 500,
                probability = TRUE,
                replace = FALSE,
                sample.fraction = .8,
                seed            = 123,
                respect.unordered.factors=TRUE)
var_imp <- as.data.frame(importance(model))
names(var_imp) <- 'imp'
var_imp$var <- row.names(var_imp)
row.names(var_imp) <- NULL
var_imp <- var_imp %>% arrange(desc(imp))
head(var_imp,10)

In [ ]:
train %>% 
  group_by(auc_win_01) %>%
  summarize(mean_tld_rat = mean(tld_rat),
           mean_sldlen_kurt = mean(sldlen_kurt),
           mean_gibbs_kurt = mean(gibbs_kurt),
           mean_sldlen_skew = mean(sldlen_skew),
           mean_gibbs_rng = mean(gibbs_rng),
           mean_ren_prp = mean(ren_prp),
           mean_sldlen_rng = mean(sldlen_rng),
           mean_pdcnt_skew = mean(pdcnt_skew),
           mean_regarpt_skew = mean(regarpt_skew)
           )

train %>% 
  group_by(auc_win_01, country_maj) %>%
  tally() %>%
  arrange(auc_win_01, desc(n)) %>% spread(country_maj, n)  %>% t()